In [467]:
from pytools import reduce
from random import random

def itob(i:int, lenght):
    return ((i >> n) & 1 for n in range(lenght))

class nd_variable:
    def __init__(self, distribution:list[float]):
        self.distribution = distribution

    def __len__(self):
        return len(self.distribution)

    def collapse(self):
        return ((random() < d) << n for n, d in enumerate(self.distribution))

    def __call__(self):
        return reduce(lambda a, b: a | b, self.collapse())
    
    def __str__(self):
        return ''.join(map(lambda i: str(i), self.collapse()))

    def __add__(self, b:'nd_variable'):
        if type(b) is float:
            return self + nd_variable([b]*len(self))
        assert len(self) == len(b)
        return collapsed(self() | b(), len(self))

    def __sub__(self, b:'nd_variable'):
        if type(b) is float:
            return self - nd_variable([b]*len(self))
        assert len(self) == len(b)
        return collapsed(self() ^ b(), len(self))

    def __mul__(self, b:'nd_variable'):
        if type(b) is float:
            return self * nd_variable([b]*len(self))
        assert len(self) == len(b)
        return collapsed(self() & b(), len(self))
    
    def __pow__(self, n:int):
        return reduce(lambda a, b: a & b, (self() for _ in range(n)))
    
    def __matmul__(self, b:'nd_variable'):
        assert len(self) == len(b)
        return reduce(lambda a, b: a|b, (a & b for a, b in zip(self.collapse(), b.collapse())))

class collapsed(nd_variable):
    def __init__(self, value, lenght):
        super().__init__([None]*lenght)
        self.value = value
    
    def __call__(self):
        return self.value

def measure(function, N, lenght, **values):
    return [g/N for g in reduce(lambda a, b: [ai + bi for ai, bi in zip(a, b)], ([*itob(function(**values).value, lenght)] for n in range(N)))]

In [531]:
x = nd_variable([0.5]*100)
y = nd_variable([0.3]*100)
sum(measure(lambda x, y: x*0.1, 10, 100, x = x, y = y))/100

0.04500000000000001

In [469]:
0.5*0.7

0.35

In [470]:
0.8*0.3

0.24